## 3.0 Импорты библиотек

In [7]:
import os
import yaml
import logging
import numpy as np
import scipy.stats as stats
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Any, Optional, Tuple, Dict
from datetime import datetime
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from scipy.stats import pearsonr, spearmanr, kurtosis, skew
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
# расширяем поле ноутбука для удобства
from IPython.display import display, HTML
display(HTML('<style>.container {width:87% !important;}</style>'))
display(HTML("<style>.output_scroll {height:auto !important; max-height:10000px !important;}</style>"))

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
# Настройки для pandas (количество отображаемых колонок)
pd.set_option('display.max_columns', 100)

In [5]:
# Определение стиля для pyplot
plt.style.use('ggplot')

In [6]:
# Текущая рабочая директория
cwd = Path().resolve()

# Поднимаемся на один уровень выше
project_root = cwd.parent

# Добавляем корень проекта в sys.path
sys.path.append(str(project_root))

# Загрузка данных из config.yaml
from src.data import downloader, loader, preprocessor, saving

# Путь к файлу config.yaml
config_path = project_root / "config" / "config.yaml"

# Загружаем конфиг
config = loader.load_config(config_path)

## 3.1. Загрузка данных

### Объявление функций

In [19]:
def data_load_preprocessed(data_type: str, config: dict) -> pd.DataFrame:
    """
    Загружает данные из pikle файла.

    Parameters:
    -----------
    data_type : str
        Тип загружаемых данных. Допустимые значения:
        - 'train' - для загрузки обучающей выборки
        - 'test' - для загрузки тестовой выборки
    config : dict
        Словарь конфигурации

    Returns:
    --------
    pd.DataFrame
        DataFrame с загруженными данными.        
    """
    processed_dir = config["data"]["processed_dir"]
    
    # if data_type == 'train':
    #     file_to_load = os.path.join('..', raw_dir, config["data"]["train_file"])
    # elif data_type == 'test':
    #     file_to_load = os.path.join('..', raw_dir, config["data"]["test_file"])
    # else:
    #     raise ValueError(f"Неизвестный тип данных: {data_type}. Допустимые значения: 'train', 'test'")
    
    file_to_load = os.path.join('..', processed_dir, config["processed_files"]["latest_cleaned"]["pkl"])
    
    
    print(f"[⧗] Загружаю данные из: {file_to_load}")
    
    try:
        df = pd.read_pickle(file_to_load)
        print(f"[✓] Данные успешно загружены. Форма: {df.shape}")
        return df
    except FileNotFoundError:
        raise FileNotFoundError(f"Файл не найден: {file_to_load}")
    except Exception as e:
        raise Exception(f"Ошибка при загрузке данных: {e}")

### Загрузка предобработанных данных

In [20]:
# Загрузка train
df_train = data_load_preprocessed(data_type='train', config=config)

[⧗] Загружаю данные из: D:\Skills\Kaggle\ml-regression_concrete-strength\data\processed\eda_data_20251009_1334.pkl
[✓] Данные успешно загружены. Форма: (781, 11)


In [21]:
# Вывод первых 5 строк тренировочного датасета
df_train.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength,W/C,Sp/C_pct
0,376.0,0.0,0.0,214.6,0.0,1003.5,762.4,3,16.28,0.570745,0.000000
1,491.0,26.0,123.0,210.0,3.9,882.0,699.0,56,59.59,0.427699,0.007943
2,250.0,0.0,95.7,187.4,5.5,956.9,861.2,3,13.82,0.749600,0.022000
3,310.0,0.0,0.0,192.0,0.0,1012.0,830.0,90,35.76,0.619355,0.000000
4,252.1,97.1,75.6,193.8,8.3,835.5,821.4,28,33.40,0.768743,0.032923
